In [1]:
import sys
sys.path.insert(1,'../functions/')
import torch
from torch import nn
import numpy as np
import data_sim
import matplotlib.pyplot as plt
import xarray as xr
import copy
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from slab_spike_model_constructor import *
from full_slab_spike_model_constructor import *
from training_func import *

In [2]:
torch.cuda.is_available()

True

In [3]:
# G_21 = xr.open_dataset("../../../Desktop/CURRENT_RESEARCH/Latent_mixture/data/G_21_subset_imputed.nc")
# G_21 = G_21.to_array().values.squeeze(0)
# G_21.shape

In [4]:
from pandas_plink import read_plink1_bin, Chunk
ABCD_full = read_plink1_bin("../../../Desktop/CURRENT_RESEARCH/Latent_mixture/data/ABCD_20220428.updated.nodups.curated.bed",
                      chunk=Chunk(nsamples=100, nvariants=None))
freq_snp = ABCD_full.mean(axis = 0)/2
sd_snp = np.sqrt(2*freq_snp*(1-freq_snp))
# a subset that have variability in the SNPs
ABCD_sub = ABCD_full[:,~sd_snp.isin(0)]
freq_snp = ABCD_sub.mean(axis = 0)/2
sd_snp = np.sqrt(2*freq_snp*(1-freq_snp))
ABCD_sub = (ABCD_sub-2*freq_snp)/sd_snp
ABCD_sub.persist()


Mapping files: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.44it/s]


<xarray.DataArray 'genotype' (sample: 11389, variant: 509151)>
dask.array<truediv, shape=(11389, 509151), dtype=float32, chunksize=(100, 509151), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object 'NDAR_INV003RTV85' ... 'NDAR_INVZZZP87KR'
  * variant  (variant) <U13 'variant0' 'variant1' ... 'variant516597'
    fid      (sample) object 'NDAR_INV003RTV85' ... 'NDAR_INVZZZP87KR'
    iid      (sample) object 'NDAR_INV003RTV85' ... 'NDAR_INVZZZP87KR'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object '1' '1' '1' '1' '1' ... '26' '26' '26' '26' '26'
    snp      (variant) object 'rs3131972' 'rs3131962' ... 'chr26:16526'
    cm       (variant) float64 0.008 0.008 0.009 0.009 0.009 ... 0.0 0.0 0.0 0.0
    pos      (variant) int32 752721 756604 779322 795222 ... 16356 16362 16526
    a0       (variant) object 'A' 'A' 'G' 'G' 'G' 'G' ... 'T' 'C' 'C' 'C' 'A'
    a1       (variant) object 'G' 'G' 'A' 'C' 'A' 'A' ... 'C' 'T' 'T' 'T' 'G'

In [5]:
ABCD_sub =  ABCD_sub.values
ABCD_sub[np.isnan(ABCD_sub)]=0

In [6]:
ABCD_sub.shape

(11389, 509151)

In [11]:
real_data_index = True
max_n_p = True
n = 11389
p = 509151
p_causal = 1000
batch_size = 1200
r_batch = batch_size/n
device = 'cuda'
if real_data_index:
    if not max_n_p:
        np.random.seed(seed = None)
        p_max = ABCD_sub.shape[1]
        n_max = ABCD_sub.shape[0]
        SNP_index = np.random.choice(p_max,size = p, replace = False)
        subject_index =  np.random.choice(n_max,size = n, replace = False)
        ABCD_sub_sub = ABCD_sub[np.ix_(subject_index, SNP_index)]
        print(ABCD_sub_sub.shape)
        Xs = [ABCD_sub_sub]
    else:
        Xs = [ABCD_sub]
    n = Xs[0].shape[0]
    p = Xs[0].shape[1]
else:
    Xs = None
print(r_batch)

0.10536482570901748


In [8]:
rho = 0.6
var = 1
n_matrix = 1
h = 0.5
beta_bias = 1
sim_class = data_sim.sim_tobit_data(n = n, p = p, p_causal = p_causal, rho = rho, var = var, n_matrix = n_matrix,h = h, bias = beta_bias, Xs = Xs, scale_lambda =None)
z, X, Xs, latent_mean, var_genetic, var_total, true_beta, y_star = sim_class.gen_data(seed = None)
#data_sim.show_sim(z, latent_mean, var_total)

In [12]:
sim_data = Sim_Dataset(X,y_star, device = device)
sim_data_loader = DataLoader(sim_data, batch_size = batch_size)
model = linear_slab_spike(p = p, n_total = n, init_pi_local = 0.5, init_pi_global = 0.1, init_beta_var = 2, init_noise_var = 0.1,
                         gumbel_softmax_temp = 1, gumbel_softmax_hard = False, 
                          a1= 0.1, a2=1, init_a3= 0.1, init_a4 = 0.1,
                          b1 = 1.1, b2 = 1.1, init_b3 = 1.1, init_b4 = 1.1, n_E = 1
                          , prior_sparsity = True, prior_sparsity_beta = True, device = device
                         ).double().to(device)
optimizer = torch.optim.Adam(
    [{'params': model.beta_mu},
     {'params': model.beta_log_var},
     {'params': model.logit_pi_local, 'lr': 0.5},
     {'params': model.log_a3},
     {'params': model.log_a4},
     {'params': model.log_b3},
     {'params': model.log_b4},
     {'params': model.bias},
     {'params': model.logit_pi_global, 'lr':0.5},
     {'params': model.beta_log_var_prior},
     {'params': model.log_var_noise},
    ],
    lr = 0.05)

In [ ]:
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.99)
t = 100 #number of moving averages
patience = 100# patience
_, result_dict = train_and_infer(model = model, optimizer = optimizer, sim_data_loader = sim_data_loader, lr_scheduler = lr_scheduler, t = t, patience = patience,X = X, plot = True, true_beta = true_beta, verbose = True)

At iteration 0, the loss is 150977340.5181643
At iteration 1000, the loss is 31939.140377132702
At iteration 2000, the loss is 15765.516291954302


In [ ]:
result_dict

In [31]:
p_causal/p

0.05

In [25]:
# torch.sum(torch.sigmoid(model.logit_pi_local)>0.9)/p
print(torch.max(torch.sigmoid(model.logit_pi_local)))
print(torch.min(torch.sigmoid(model.logit_pi_local)))
# torch.min(model.beta_mu)
# torch.max(model.beta_mu)
torch.exp(model.beta_log_var)
print(torch.mean(torch.exp(model.beta_log_var)))

tensor(1.0000, dtype=torch.float64, grad_fn=<MaxBackward1>)
tensor(6.4280e-09, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(0.4833, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [9]:
c = torch.exp(best_model.log_c).detach().numpy()
d = torch.exp(best_model.log_d).detach().numpy()
# c = 10
# d = 10
print(c)
print(d)
posterior = np.random.beta(c,d,size = (5000,))
np.median(posterior)
print(np.quantile(posterior, q = 0.025))
print(np.quantile(posterior, q = 0.975))
plt.hist(posterior)
plt.show()

NameError: name 'best_model' is not defined